### Webtoon 회차별 정보
- 웹툰명 : title
- 회차 : no
- 웹툰 회차의 기본키(ID값)
- 회차별 썸네일링크

In [19]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
# 함수화
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['title'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['main_link'] = []
    dic['image_link'] = []
    dic['update_date'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
    for keyword in keywords:
        dic['title'].append(keyword.select_one("td > a > strong").text)
    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['author'].append(keyword.select_one("dl > dd > a").text)
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
        dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    
    return df

In [4]:
genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]

In [5]:
df = naver_webtoon("episode")

In [6]:
for genre in genres[1:]:
    print(genre)
    sample = naver_webtoon(genre)
    df = pd.merge(df,sample, how='outer')
df.reset_index(inplace=True, drop=True)
df.tail()

omnibus
story
daily
comic
fantasy
action
drama
pure
sensibility
thrill
historical
sports


,unique_id,title,author,total_score,main_link,image_link,update_date,episode,omnibus,story,daily,comic,fantasy,action,drama,pure,sensibility,thrill,historical,sports
739,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.23,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,pure,sensibility,NaN,NaN,NaN
740,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,NaN,NaN,story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,336945,길에서 만나다,쥬드 프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,NaN,NaN,story,NaN,NaN,NaN,NaN,drama,NaN,sensibility,NaN,NaN,NaN
742,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,NaN,NaN,story,NaN,NaN,fantasy,NaN,drama,NaN,NaN,NaN,NaN,NaN
743,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,NaN,NaN,story,NaN,comic,NaN,NaN,drama,NaN,NaN,NaN,NaN,NaN


In [7]:
def make_genre(df):
    genres = ["episode","omnibus","story","daily","comic","fantasy","action","drama","pure","sensibility","thrill","historical","sports"]
    df.fillna("", inplace=True)

    result = []
    for i in range(len(df[genres])):
        ls = list(filter(lambda value: value != "", df[genres].loc[i].values))
        result.append(ls)

    df['genre'] = np.array(["/".join(ls) for ls in result])
    df.drop(columns=genres, axis=1, inplace=True)
    return df

In [8]:
df = make_genre(df)
df.tail()

,unique_id,title,author,total_score,main_link,image_link,update_date,genre
739,622643,진눈깨비 소년,쥬드프라이데이,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.23,story/pure/sensibility
740,703840,한국만화 1990,웹툰작가,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.05.17,story
741,336945,길에서 만나다,쥬드 프라이데이,9.75,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2013.01.17,story/drama/sensibility
742,515883,사또(Satto),최윤진,9.80,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2015.12.29,story/fantasy/drama
743,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama


In [10]:
titleid = df.unique_id.values

In [94]:
dic = {}
dic['title'] = []
for tid in titleid:
    for num in range(1,201):
        url = "https://comic.naver.com/webtoon/list.nhn?titleId="+ tid +"&page=" + num
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        
        # title
        dic['title'].append(df[df.unique_id == tid].title[0])
        
        episodes = dom.select("#content > table.viewList > tr:nth-of-type(2), tr:nth-of-type(3), tr:nth-of-type(4), tr:nth-of-type(5), tr:nth-of-type(6), tr:nth-of-type(7), tr:nth-of-type(8), tr:nth-of-type(9)")
        
        for episode in episodes:
            link = episode.select_one("td > a")
            dic['image_link'].append(link.get('src'))
        images.append(episode.get('src') [for episode in episodes])
        # image link
        

SyntaxError: invalid syntax (<ipython-input-94-e1540c49117c>, line 14)

In [111]:
url = "https://comic.naver.com/webtoon/list.nhn?titleId=651673&page=2"
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
episodes = dom.select("#content > table.viewList > tr:nth-of-type(2), tr:nth-of-type(3), tr:nth-of-type(4), tr:nth-of-type(5), tr:nth-of-type(6), tr:nth-of-type(7), tr:nth-of-type(8), tr:nth-of-type(9), tr:nth-of-type(10), tr:nth-of-type(11)")
len(episodes)
for episode in episodes:
    print(episode.get('src'))

None
None
None
None
None
None
None
None
None


In [88]:
dom.select("#content > table.viewList > tr")

[<tr class="band_banner">
 <td colspan="4">
 <a href="#" onclick="callWebAdBannerClick('/api/stat/gameBanner.nhn?type=CLICK&amp;target=ARTICLE_LIST&amp;bannerId=128', 'https://yumiscells.superbcorp.com/');clickcr(this,'lst.adbanner','','',event);return false;">
 <img alt="AD 배너" height="44" src="https://naverwebtoon-phinf.pstatic.net/20180817_257/1534482725737dKSDa_JPEG/upload_8025313691756401672.JPEG" width="694"/>
 </a>
 </td>
 </tr>, <tr>
 <td>
 <a href="/webtoon/detail.nhn?titleId=651673&amp;no=325&amp;weekday=wed" onclick="clickcr(this,'lst.img','651673','325',event)">
 <img alt="322화 멋있는 포인트" height="41" onerror="this.src='https://static-comic.pstatic.net/staticImages/COMICWEB/NAVER/img/common/non71_41.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/651673/325/thumbnail_202x120_71204667-97db-40d6-be2f-ad1e62e399bc.jpg" title="322화 멋있는 포인트" width="71"/>
 <span class="mask"></span>
 </a>
 </td>
 <td class="title">
 <a href="/webtoon/detail.nhn?titleId=651673&amp;no=325&am